In [ ]:
%matplotlib inline
import xarray as xr
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import cartopy
import cartopy.geodesic
import shapely
import cartopy.crs as ccrs
import cartopy.feature
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.axes as maxes
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def plot_background_map():
    lat1, lat2, lon1, lon2 = 47.093, 50.485,  4.40, 14.368
    rp = ccrs.RotatedPole(pole_longitude=-170,
                          pole_latitude=40,
                          globe=ccrs.Globe(semimajor_axis=6370000,
                                           semiminor_axis=6370000))
    pc = ccrs.PlateCarree()

    ax = plt.axes(projection=rp)
    resol = '50m'  # use data at this scale
    ax.coastlines(resol, linewidth=0.8)
    bodr = cartopy.feature.NaturalEarthFeature(category='cultural', name='admin_0_boundary_lines_land', scale=resol, facecolor='none', alpha=0.7)
    land = cartopy.feature.NaturalEarthFeature('physical', 'land', scale=resol, edgecolor='k', facecolor=cartopy.feature.COLORS['land'])
    ocean = cartopy.feature.NaturalEarthFeature('physical', 'ocean', scale=resol, edgecolor='none', facecolor=cartopy.feature.COLORS['water'])
    ax.add_feature(land, facecolor='beige', alpha=0.1)
    ax.add_feature(ocean, linewidth=0.2, alpha=0.1)
    ax.add_feature(bodr, linestyle='-', edgecolor='k', alpha=1)
#     ax.set_extent((-3.74723296300748, 3.5357004661247644, -3.205501464193482, 5.256409405113779), crs=rp)
    return ax

def colorbar_vert(mappable, fig, ax, labels=None, orient='vertical'):
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05, axes_class=maxes.Axes)
    if labels is None:
        return fig.colorbar(mappable, cax=cax, orientation=orient, format='%.1f')
    else:
        return plt.colorbar(mappable, cax=cax, ticks=labels, orientation=orient)

In [ ]:
def read_psp(directory):
    xdata_mems = []
    for mem in range(1,21):
        try:
            filename = '/project/meteo/work/M.Puh/PhD/trial_dwd/'+str(directory)+'/'+str(directory)+'00_pspa5_mem{0}.nc'.format(mem)
            xdata_mems.append(xr.open_dataset(filename))
        except:
            filename = '/project/meteo/work/M.Puh/PhD/trial_dwd/'+str(directory)+'/'+str(directory)+'00_psp_mem{0}.nc'.format(mem)
            xdata_mems.append(xr.open_dataset(filename))
    xdata = xr.concat(xdata_mems, "ens")
    return xdata

def read_ref(directory):
    xdata_mems = []
    for mem in range(1,21):
        filename = '/project/meteo/work/M.Puh/PhD/trial_dwd/tau_c_data/'+str(directory)+'/'+str(directory)+'00_ref_mem{0}.nc'.format(mem)
        xdata_mems.append(xr.open_dataset(filename))
    xdata = xr.concat(xdata_mems, "ens")
    return xdata

def set_domain(prec, grid):
    lo1, lo2, la1, la2 = (-3.81372049983167, 2.9758399765944414, -2.8200989408733395, 4.073576825913025)
    prec = np.where(grid[0]<lo2, prec, np.nan)
    prec = np.where(grid[0]>=lo1, prec, np.nan)
    prec = np.where(grid[1]<=la2, prec, np.nan)
    prec = np.where(grid[1]>la1, prec, np.nan)
    return prec

def d64todt(dt64):
    ts = (dt64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    return dt.datetime.utcfromtimestamp(ts)

def cumsub_4d(a):
    z = a.copy()
    z[:,1:,:,:] -= z[:,:-1,:,:].copy()
    return z

In [ ]:
day = '20210609'
xdata_ref = read_ref(day)

In [ ]:
xdata_ref

In [ ]:
xdata_ref['TOT_PREC']

In [ ]:
tot_prec_ref = xdata_ref['TOT_PREC'].values
type(tot_prec_ref)

In [ ]:
tot_prec_ref.shape

# Maps

In [ ]:
tot_prec_ref_24h = tot_prec_ref[:,24,:,:]

In [ ]:
tot_prec_ref_24h.shape

In [ ]:
fig = plt.figure(figsize=(7,12))
ax = plot_background_map()
levels=[0.1, 0.5, 1, 5, 10, 20, 50, 100]
im = ax.contourf(xdata_ref['rlon'], xdata_ref['rlat'], tot_prec_ref_24h[0], cmap='jet', extend='max', alpha=1, levels=levels, norm=LogNorm())
cbar = colorbar_vert(im, fig, ax)
cbar.minorticks_off()
ax.set_title('Accumulated 24h precipitation, {2}-{1}-{0}, \n member 0'.format(day[:4], day[4:6], day[6:8]), fontsize=13)
plt.show()

In [ ]:
for mem in range(20):
    fig = plt.figure(figsize=(7,12))
    ax = plot_background_map()
    levels=[0.1, 0.5, 1, 5, 10, 20, 50, 100]
    im = ax.contourf(xdata_ref['rlon'], xdata_ref['rlat'], tot_prec_ref_24h[mem], cmap='jet', extend='max', alpha=1, levels=levels, norm=LogNorm())
    cbar = colorbar_vert(im, fig, ax)
    cbar.minorticks_off()
    ax.set_title('Accumulated 24h precipitation, {2}-{1}-{0}, \n member {3}'.format(day[:4], day[4:6], day[6:8], mem), fontsize=13)
    plt.show()

In [ ]:
fig = plt.figure(figsize=(7,12))
ax = plot_background_map()
levels=[0.1, 0.5, 1, 5, 10, 20, 50, 100]
im = ax.contourf(xdata_ref['rlon'], xdata_ref['rlat'], tot_prec_ref_24h[:].mean(axis=0), cmap='jet', extend='max', alpha=1, levels=levels, norm=LogNorm())
cbar = colorbar_vert(im, fig, ax)
cbar.minorticks_off()
ax.set_title('Accumulated 24h precipitation, {2}-{1}-{0}, \n ensemble mean'.format(day[:4], day[4:6], day[6:8]), fontsize=13)
plt.show()

# Time series

In [ ]:
tot_prec_ref.shape

In [ ]:
tot_prec_ref.mean(axis=(2,3)).shape

In [ ]:
plt.plot(tot_prec_ref.mean(axis=(2,3)).T)
plt.show()

In [ ]:
band_ref=np.quantile(np.nanmean(tot_prec_ref, axis=(2,3)), (0.25,0.5,0.75), axis=0)
plt.fill_between(range(25), y1=band_ref[0,:], y2=band_ref[2,:], alpha=0.2, zorder=0, color='red')
plt.plot(tot_prec_ref.mean(axis=(2,3)).T, color='k', alpha=0.1)
plt.plot(tot_prec_ref.mean(axis=(0,2,3)), color='k', alpha=1)
plt.title('Domain averaged accumulated precipitation, \n {2}-{1}-{0}'.format(day[:4], day[4:6], day[6:8]), fontsize=13)
plt.xlabel('Forecast lead time [h]')
plt.ylabel('Precipitation [mm]')
plt.show()

In [ ]:
prec_ref = cumsub_4d(tot_prec_ref)
prec_ref.shape

In [ ]:
band_ref=np.quantile(np.nanmean(prec_ref, axis=(2,3)), (0.25,0.5,0.75), axis=0)
plt.fill_between(range(25), y1=band_ref[0,:], y2=band_ref[2,:], alpha=0.2, zorder=0, color='red')
plt.plot(prec_ref.mean(axis=(2,3)).T, color='k', alpha=0.1)
plt.plot(prec_ref.mean(axis=(0,2,3)), color='k', alpha=1)
plt.title('Domain averaged hourly precipitation, \n {2}-{1}-{0}'.format(day[:4], day[4:6], day[6:8]), fontsize=13)
plt.xlabel('Forecast lead time [h]')
plt.ylabel('Precipitation [mm]')
plt.show()

In [ ]:
times = xdata_ref['time'].values
times_labels = [d64todt(t).strftime("%H") for t in times]

In [ ]:
band_ref=np.quantile(np.nanmean(prec_ref, axis=(2,3)), (0.25,0.5,0.75), axis=0)
plt.fill_between(times, y1=band_ref[0,:], y2=band_ref[2,:], alpha=0.2, zorder=0, color='red')
plt.plot(times, prec_ref.mean(axis=(2,3)).T, color='k', alpha=0.1)
plt.plot(times, prec_ref.mean(axis=(0,2,3)), color='k', alpha=1)
plt.xticks(times[::3], times_labels[::3])
plt.title('Domain averaged hourly precipitation, \n {2}-{1}-{0}'.format(day[:4], day[4:6], day[6:8]), fontsize=13)
plt.xlabel('UTC time [h]')
plt.ylabel('Precipitation [mm]')
plt.show()

# Domain selection

In [ ]:
tot_prec_ref.shape

In [ ]:
lats = np.asarray(xdata_ref['rlat'].values)
lons = np.asarray(xdata_ref['rlon'].values)
lon_grid, lat_grid = np.meshgrid(lons, lats)

In [ ]:
tot_prec_ref_DE = set_domain(xdata_ref['TOT_PREC'].values, (lon_grid, lat_grid))
tot_prec_ref_DE.shape

In [ ]:
fig = plt.figure(figsize=(7,12))
ax = plot_background_map()
levels=[0.1, 0.5, 1, 5, 10, 20, 50, 100]
im = ax.contourf(xdata_ref['rlon'], xdata_ref['rlat'], tot_prec_ref_DE[:,-1,:,:].mean(axis=0), cmap='jet', extend='max', alpha=1, levels=levels, norm=LogNorm())
cbar = colorbar_vert(im, fig, ax)
cbar.minorticks_off()
ax.set_title('Accumulated 24h precipitation, {2}-{1}-{0}, \n ensemble mean'.format(day[:4], day[4:6], day[6:8]), fontsize=13)
plt.show()

In [ ]:
prec_ref_DE = cumsub_4d(tot_prec_ref_DE)
prec_ref_DE.shape

In [ ]:
times = xdata_ref['time'].values
times_labels = [d64todt(t).strftime("%H") for t in times]

In [ ]:
band_ref=np.quantile(np.nanmean(prec_ref_DE, axis=(2,3)), (0.25,0.5,0.75), axis=0)
plt.fill_between(times, y1=band_ref[0,:], y2=band_ref[2,:], alpha=0.2, zorder=0, color='red')
plt.plot(times, np.nanmean(prec_ref_DE, axis=(2,3)).T, color='k', alpha=0.1)
plt.plot(times, np.nanmean(prec_ref_DE, axis=(0,2,3)), color='k', alpha=1)
plt.xticks(times[::3], times_labels[::3])
plt.title('Domain averaged hourly precipitation, \n {2}-{1}-{0}'.format(day[:4], day[4:6], day[6:8]), fontsize=13)
plt.xlabel('UTC time [h]')
plt.ylabel('Precipitation [mm]')
plt.show()

# Radar data

In [ ]:
from enstools.io import read
import wradlib as wrl

In [ ]:
def get_radar_data(day):
    loc='/archive/meteo/external-obs/dwd/radar/radolan_RY_gridded-native_rolling/latest/'
    rydata=!ls -d {loc}/radolan_EY_{day}*.nc
    data=read(rydata)

    def cutry(domain,datain):
        ds_prcp=datain
        if domain == 'full':
            ds_prcp=ds_prcp.where( ds_prcp.lat > 43.19, drop=True)
            ds_prcp=ds_prcp.where( ds_prcp.lat <= 58.09, drop=True)
            ds_prcp=ds_prcp.where( ds_prcp.lon >= -3.99,drop=True)
            ds_prcp=ds_prcp.where( ds_prcp.lon < 20.32,drop=True)
        if domain == 'DE':
            ds_prcp=ds_prcp.where( ds_prcp.lat > 47.093, drop=True)
            ds_prcp=ds_prcp.where( ds_prcp.lat <= 53.95, drop=True)
            ds_prcp=ds_prcp.where( ds_prcp.lon >= 4.40,drop=True)
            ds_prcp=ds_prcp.where( ds_prcp.lon < 14.368,drop=True)
        return ds_prcp

    de_ry=cutry("DE",data)
    full_ry=cutry("full",data)

    de_me=np.nanmean(de_ry["pr"].values, axis=(1,2))*300
    hrly_de=np.reshape(de_me, (24, 12))
    hrly_de_sum=np.sum(hrly_de,axis=1)

    return hrly_de_sum, np.nansum(np.asarray(full_ry['pr']), axis=0)

In [ ]:
hrly_de_sum, radar_sum = get_radar_data(day)

In [ ]:
radolan_grid_ll = wrl.georef.get_radolan_grid(900,900, wgs84=True)

In [ ]:
rad_lon = radolan_grid_ll[:,:,0]
rad_lat = radolan_grid_ll[:,:,1]
lon_grid, lat_grid = rad_lon, rad_lat

In [ ]:
radar_sum.shape

In [ ]:
fig = plt.figure(figsize=(7,12))
ax = plot_background_map()
levels=[0.1, 0.5, 1, 5, 10, 20, 50, 100]

im = ax.contourf(lon_grid.T, lat_grid.T, radar_sum.T*300, cmap='jet', extend='max', alpha=1, levels=levels, norm=LogNorm(), transform=ccrs.PlateCarree())

cbar = colorbar_vert(im, fig, ax)
# cbar.ax.tick_params(labelsize=12)
cbar.minorticks_off()

ax.set_title('Accumulated 24h precipitation, {2}-{1}-{0} \n radar observations'.format(day[:4], day[4:6], day[6:8]), fontsize=13)
# plt.savefig('plots/radar/radar0906ry.png', bbox_inches="tight", dpi=200)

In [ ]:
hrly_de_sum

In [ ]:
plt.plot(times[1:], hrly_de_sum, color='tab:blue', linewidth=5, alpha=0.2)
plt.xticks(times[::3], times_labels[::3])
plt.title('Domain averaged hourly precipitation, \n {2}-{1}-{0} (radar data)'.format(day[:4], day[4:6], day[6:8]), fontsize=13)
plt.xlabel('UTC time [h]')
plt.ylabel('Precipitation [mm]')
plt.show()

In [ ]:
band_ref=np.quantile(np.nanmean(prec_ref_DE, axis=(2,3)), (0.25,0.5,0.75), axis=0)
plt.fill_between(times, y1=band_ref[0,:], y2=band_ref[2,:], alpha=0.2, zorder=0, color='grey')
plt.plot(times, np.nanmean(prec_ref_DE, axis=(0,2,3)), color='k', alpha=1, label='ref')
plt.plot(times[1:], hrly_de_sum, color='tab:blue', linewidth=5, alpha=0.2, label='radar')
plt.xticks(times[::3], times_labels[::3])
plt.title('Domain averaged hourly precipitation, \n {2}-{1}-{0}'.format(day[:4], day[4:6], day[6:8]), fontsize=13)
plt.xlabel('UTC time [h]')
plt.ylabel('Precipitation [mm]')
plt.legend()
plt.show()

In [ ]:
day = '20210609'
xdata_psp = read_psp(day)
lats = np.asarray(xdata_psp['rlat'].values)
lons = np.asarray(xdata_psp['rlon'].values)
lon_grid, lat_grid = np.meshgrid(lons, lats)
tot_prec_psp_DE = set_domain(xdata_psp['TOT_PREC'].values, (lon_grid, lat_grid))
prec_psp_DE = cumsub_4d(tot_prec_psp_DE)

In [ ]:
band_ref=np.quantile(np.nanmean(prec_ref_DE, axis=(2,3)), (0.25,0.5,0.75), axis=0)
band_psp=np.quantile(np.nanmean(prec_psp_DE, axis=(2,3)), (0.25,0.5,0.75), axis=0)
plt.fill_between(times, y1=band_ref[0,:], y2=band_ref[2,:], alpha=0.2, zorder=0, color='grey')
plt.plot(times, np.nanmean(prec_ref_DE, axis=(0,2,3)), color='k', alpha=1, label='ref')
plt.fill_between(times, y1=band_psp[0,:], y2=band_psp[2,:], alpha=0.2, zorder=0, color='tab:red')
plt.plot(times, np.nanmean(prec_psp_DE, axis=(0,2,3)), color='red', alpha=1, label='psp')
plt.plot(times[1:], hrly_de_sum, color='tab:blue', linewidth=5, alpha=0.2, label='radar')
plt.xticks(times[::3], times_labels[::3])
plt.title('Domain averaged hourly precipitation, \n {2}-{1}-{0}'.format(day[:4], day[4:6], day[6:8]), fontsize=13)
plt.xlabel('UTC time [h]')
plt.ylabel('Precipitation [mm]')
plt.legend()
plt.show()

# Task 1

Write a function called "plotting_function" that takes as argument a string of format 'YYYYMMDD' and produces the figure above. \
The function has to read the data (reference, psp and radar) for any day in June, July and August 2021.

In [ ]:
def plotting_function(day_yyyymmdd):
    
    # WRITE 
    # FUNCTION
    # HERE
    
    return 0

In [ ]:
plt.rcParams['figure.figsize'] = [5.5,4]

day = '20210609'

plotting_function(day)

plt.savefig('/project/meteo/work/M.Puh/Hands_on/plots/time_series_{0}.png'.format(day), bbox_inches="tight", dpi=200)

# Task 2

Now write a loop over a whole month, producing a figure like above for every day.

In [ ]:
plt.rcParams['figure.figsize'] = [5.5,4]

for day in ...:

    plotting_function(day)

    plt.savefig('/project/meteo/work/M.Puh/Hands_on/plots/time_series_{0}.png'.format(day), bbox_inches="tight", dpi=200)

# Task 3

Produce maps of the total daily accumulated precipitation (ensemble mean) of the reference and psp experiments for the whole period (choose a month).

!! **REMINDER**: uncomment "set_extent" line in "plot_background_map" function before.

In [ ]:
fig = plt.figure(figsize=(7,12))

exp = 'ref'
# exp = 'psp'

for day in ...:

    

    plt.savefig('/project/meteo/work/M.Puh/Hands_on/plots/map24h_{0}_{1}.png'.format(exp, day), bbox_inches="tight", dpi=200)

# Task 4

Produce maps of the total daily accumulated precipitation (for all the single members) of the reference and psp experiments for the whole period (choose a month).

In [ ]:
fig = plt.figure(figsize=(7,12))

exp = 'ref'
# exp = 'psp'

for day in ...:
    
    for member in range(20):

    

        plt.savefig('/project/meteo/work/M.Puh/Hands_on/plots/map24h_{0}_{1}_mem{2}.png'.format(exp, day, member), bbox_inches="tight", dpi=200)

# Task 5

Produce maps of the total daily accumulated precipitation estimated from radar observations for the whole period (choose a month).

In [ ]:
fig = plt.figure(figsize=(7,12))

exp = 'ref'
# exp = 'psp'

for day in ...:

    

    plt.savefig('/project/meteo/work/M.Puh/Hands_on/plots/radar24h_{0}_{1}.png'.format(exp, day), bbox_inches="tight", dpi=200)